get austin twitter from 2TB covid-19 twitter dataset

(do not run this code, unless you want to get the data of other cities)

In [ ]:
! pip install jsonlines

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
covidDataPath = 'drive/MyDrive/COVID-19 Tweets Dataset/'
month = '2021-07'
austinCoordinates = [[[-97.928935, 30.127892],
     [-97.5805133, 30.127892],
     [-97.5805133, 30.5187994],
     [-97.928935, 30.5187994]]]
# left bound
leftAustin = austinCoordinates[0][0][0]
# right bound
rightAustin = austinCoordinates[0][1][0]
# upper bound
upAustin = austinCoordinates[0][2][1]
# lower bound
downAustin = austinCoordinates[0][0][1]
# read the data
import os
import gzip
import json
from tqdm import tqdm
import jsonlines

total_n_tweet = 0
tweet_userlocation = 0
tweet_placeFullName = 0
tweet_boxPlace = 0
tweet_pointPlace = 0

writerUserLocation = jsonlines.Writer(open('drive/MyDrive/austinCovid/202104-/'+month+'austinUserLocation.jsonl', 'w'))
writerTweetLocation = jsonlines.Writer(open('drive/MyDrive/austinCovid/202104-/'+month+'austinTweetLocation.jsonl', 'w'))
for filename in tqdm(sorted(os.listdir(covidDataPath+month))):
  if filename.endswith('.gz'):
    try:
      with gzip.open(covidDataPath+month+'/'+filename, mode="rt") as f:
        for line in f:
          try:
            tweet = json.loads(line)
            total_n_tweet += 1
            if 'austin' in  tweet['user']['location'].lower() and  'austintown' not in tweet['user']['location'].lower():
              writerUserLocation.write(tweet)
              tweet_userlocation += 1
            if  tweet['place'] != None:
              if 'austin' in tweet['place']['full_name'].lower() and 'austintown' not in tweet['place']['full_name'].lower():
                writerTweetLocation.write(tweet)
                tweet_placeFullName += 1
                # area match
              elif  tweet['place']['bounding_box']!=None and tweet['place']['bounding_box']['coordinates'][0][0][0] > leftAustin and   tweet['place']['bounding_box']['coordinates'][0][1][0] < rightAustin  and tweet['place']['bounding_box']['coordinates'][0][2][1] < upAustin and tweet['place']['bounding_box']['coordinates'][0][0][1] > downAustin:
                writerTweetLocation.write(tweet)
                tweet_boxPlace += 1
              # point match
              elif tweet['coordinates'] != None and (leftAustin <= tweet['coordinates']['coordinates'][0] <= rightAustin and downAustin <= tweet['coordinates']['coordinates'][1] <= upAustin):
                writerTweetLocation.write(tweet)
                tweet_pointPlace += 1
          except Exception as e:
            print(e)
            print(filename)
            print(line)
            continue
    except Exception as e:
      print(e)
      print(filename)
      continue

print('total_n_tweet',total_n_tweet)
print('tweet_userlocation' ,tweet_userlocation)
print('tweet_placeFullName',tweet_placeFullName)
print('tweet_boxPlace', tweet_boxPlace)
print('tweet_pointPlace' , tweet_pointPlace)